#This notebook can be divided to three parts.
1. Openmm installation and a test simulation
2. Openmm simulation with Amber input, i.e., a prmtop file, and a (inp)crd file
3. Openmm simulation with VMD/Charmm input, i.e., a psf file and a pdb file.

Note: The method of how to generate Amber or VMD/Charmm input is not incluede here, you are supposed to be confident with at least one of them before using this notebook.

Created by quantaosun@gmail.com, based on the official openmm documentation in https://openmm.org/documentation 

A potential update would include method of standard free energy of protein ligand binding.

## Note, the next 3htb, example, i.e., the first OpenMM simulation is just fro testing purpose, to verify our simulation software is working. For a more proper simulation you should use the following two, i.e., the Amber input version or the VMD/Charmm input verion.

In [1]:
#@title  Install Dependencies
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local
#! conda install -c rdkit rdkit -y
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

--2021-11-15 04:52:19--  https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8303, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85055499 (81M) [application/x-sh]
Saving to: ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’

Miniconda3-py37_4.8 100%[===================>]  81.12M   161MB/s    in 0.5s    

2021-11-15 04:52:20 (161 MB/s) - ‘Miniconda3-py37_4.8.2-Linux-x86_64.sh’ saved [85055499/85055499]

PREFIX=/usr/local
Unpacking payload ...
Solving environment: - \ done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - _libgcc_mutex==0.1=main
    - asn1crypto==1.3.0=py37_0
    - ca-certificates==2020.1.1=0
    - certifi==2019.11.28=py37_0
    - cffi==1.14.0=py37h2e261b9_0
    - chardet==3.0.4=py37_1003
    - conda-package-handling==1.6.0=py37h

In [2]:
#@title Install OpenMM
!yes|conda install -c conda-forge openmm

Solving environment: / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.2
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - openmm


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _libgcc_mutex-0.1          |      conda_forge           3 KB  conda-forge
    _openmp_mutex-4.5          |            1_gnu          22 KB  conda-forge
    ca-certificates-2021.10.8  |       ha878542_0         139 KB  conda-forge
    certifi-2021.10.8          |   py37h89c1867_1         145 KB  conda-forge
    conda-4.10.3               |   py37h89c1867_3         3.1 MB  conda-forge
    cudatoolkit-1

In [3]:
#@title Install Pdbfixer, you could skip this step
!yes|conda install -c conda-forge pdbfixer

Solving environment: \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - pdbfixer


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    pdbfixer-1.8.1             |     pyh6c4a22f_0         498 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         498 KB

The following NEW packages will be INSTALLED:

  pdbfixer           conda-forge/noarch::pdbfixer-1.8.1-pyh6c4a22f_0


Proceed ([y]/n)? 

pdbfixer-1.8.1       | 498 KB    | : 100% 1.0/1 [00:00<00:00,  5.86it/s]
Preparing transaction: | done
Verifying transaction: - done
Executing transaction: | / - \ done


In [4]:
#@title Verify OpenMM installation works or not.
!python -m openmm.testInstallation


OpenMM Version: 7.6
Git Revision: ad113a0cb37991a2de67a08026cf3b91616bafbe

There are 4 Platforms available:

1 Reference - Successfully computed forces
2 CPU - Successfully computed forces
3 CUDA - Successfully computed forces
4 OpenCL - Successfully computed forces

Median difference in forces between platforms:

Reference vs. CPU: 6.29811e-06
Reference vs. CUDA: 6.726e-06
CPU vs. CUDA: 7.4071e-07
Reference vs. OpenCL: 6.76294e-06
CPU vs. OpenCL: 8.04873e-07
CUDA vs. OpenCL: 2.6522e-07

All differences are within tolerance.


In [5]:
#@title Prepare the raw PDB structure with PDBfixer
!pdbfixer --pdbid=3htb --output=myfile.pdb --add-residues 
from pdbfixer import PDBFixer
from simtk.openmm.app import PDBFile
fixer = PDBFixer(filename='myfile.pdb')
fixer.findMissingResidues()
fixer.findNonstandardResidues()
fixer.replaceNonstandardResidues()
fixer.removeHeterogens(True)
fixer.findMissingAtoms()
fixer.addMissingAtoms()
fixer.addMissingHydrogens(7.0)
fixer.addSolvent(fixer.topology.getUnitCellDimensions())
PDBFile.writeFile(fixer.topology, fixer.positions, open('input.pdb', 'w'))

In [6]:
#@title This is not a proper simulation, just for testing purpose
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout

pdb = PDBFile('input.pdb')
forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME, nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinIntegrator(300*kelvin, 1/picosecond, 0.002*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True, potentialEnergy=True, temperature=True))
simulation.step(100000)

#"Step","Potential Energy (kJ/mole)","Temperature (K)"
1000,-393063.51718441583,250.46731509114073
2000,-374199.76718441583,285.3240328958211
3000,-368188.51718441583,298.37104468228387
4000,-367297.14218441583,299.47165154330986
5000,-366620.89218441583,298.3527292026586
6000,-366715.14218441583,301.62707700364314
7000,-367285.01718441583,302.0283776377216
8000,-366829.01718441583,299.7236989110745
9000,-366172.01718441583,303.0446388381504
10000,-366042.01718441583,301.0553184393321
11000,-367739.51718441583,302.38660896385915
12000,-367155.51718441583,299.7437662461676
13000,-367114.01718441583,301.57057719454707
14000,-365743.76718441583,299.2018057875568
15000,-365781.26718441583,298.8220588027557
16000,-367144.14218441583,304.09720076394484
17000,-365583.26718441583,303.6682191087738
18000,-365749.01718441583,299.29942145076484
19000,-366929.26718441583,299.9396039003095
20000,-367159.26718441583,296.4246116746089
21000,-367366.64218441583,300.55243520451455
22000,-367227.0171844

#This is an example of using Amber input, this could be a proper simulation, as long as your Amber input is good.✈

In [ ]:
#@title This script was generated by OpenMM-Setup on 2021-11-15.

from simtk.openmm import *
from simtk.openmm.app import *
from simtk.unit import *

# Input Files

prmtop = AmberPrmtopFile('top5.prmtop')
inpcrd = AmberInpcrdFile('top5.crd')

# System Configuration

nonbondedMethod = PME
nonbondedCutoff = 1.0*nanometers
ewaldErrorTolerance = 0.0005
constraints = HBonds
rigidWater = True
constraintTolerance = 0.000001
hydrogenMass = 1.5*amu

# Integration Options

dt = 0.004*picoseconds
temperature = 300*kelvin
friction = 1.0/picosecond
pressure = 1.0*atmospheres
barostatInterval = 25

# Simulation Options

steps = 1000000
equilibrationSteps = 1000
platform = Platform.getPlatformByName('CUDA')
platformProperties = {'Precision': 'single'}
dcdReporter = DCDReporter('trajectory.dcd', 10000)
dataReporter = StateDataReporter('log.txt', 1000, totalSteps=steps,
    step=True, speed=True, progress=True, potentialEnergy=True, temperature=True, separator='\t')
checkpointReporter = CheckpointReporter('checkpoint.chk', 10000)

# Prepare the Simulation

print('Building system...')
topology = prmtop.topology
positions = inpcrd.positions
system = prmtop.createSystem(nonbondedMethod=nonbondedMethod, nonbondedCutoff=nonbondedCutoff,
    constraints=constraints, rigidWater=rigidWater, ewaldErrorTolerance=ewaldErrorTolerance, hydrogenMass=hydrogenMass)
system.addForce(MonteCarloBarostat(pressure, temperature, barostatInterval))
integrator = LangevinMiddleIntegrator(temperature, friction, dt)
integrator.setConstraintTolerance(constraintTolerance)
simulation = Simulation(topology, system, integrator, platform, platformProperties)
simulation.context.setPositions(positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)

# Minimize and Equilibrate

print('Performing energy minimization...')
simulation.minimizeEnergy()
print('Equilibrating...')
simulation.context.setVelocitiesToTemperature(temperature)
simulation.step(equilibrationSteps)

# Simulate

print('Simulating...')
simulation.reporters.append(dcdReporter)
simulation.reporters.append(dataReporter)
simulation.reporters.append(checkpointReporter)
simulation.currentStep = 0
simulation.step(steps)

Building system...
Performing energy minimization...
Equilibrating...
Simulating...


In [ ]:
#@title This is the code example taken from Openmm's user guide document.

from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

prmtop = AmberPrmtopFile('SYS_gaff2.prmtop')
inpcrd = AmberInpcrdFile('SYS_gaff2.crd')
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1*nanometer,
        constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(prmtop.topology, system, integrator)
simulation.context.setPositions(inpcrd.positions)
if inpcrd.boxVectors is not None:
    simulation.context.setPeriodicBoxVectors(*inpcrd.boxVectors)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('amber_output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)

#This is an example of using VMD/Charmm input. This is my personal favourate, since I use VMD exclusively.✈⛹ Just be reminded the topolog and parameter stuff sometime easy to go wrong.

In [ ]:
#@title 6 👋Download Charmm Force Field
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/top_all36_prot.prm 
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/top_all36_prot.rtf 
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/toppar_water_ions.mod.str
!wget https://raw.githubusercontent.com/quantaosun/NAMD-MD/main/toppar_water_ions.str

In [ ]:
#@title Be ware of topology of small molecule. Upload all the rtf and prm you would run a normal NAMD simulation, to here in order to run an OpenMM simulation.

In [ ]:
#@title  This is a simulation with PSF and PDB files generated from VMD, as an input for OpenMM
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout, exit, stderr

psf = CharmmPsfFile('ionized.psf')
pdb = PDBFile('ionized.pdb')
params = CharmmParameterSet('top_all36_prot.rtf', 'top_all36_prot.prm','toppar_water_ions.mod.str','toppar_water_ions.str','ligand_ligpargen.prm','ligand_ligpargen.rtf')
system = psf.createSystem(params, nonbondedMethod=NoCutoff,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(psf.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(DCDeporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)